# 強化学習（LLM用）の定義

In [1]:
from trl import AutoModelForCausalLMWithValueHead
import torch
from transformers import AutoModelForCausalLM
from transformers import pipeline, AutoTokenizer

model_name = 'gpt2'
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
value_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')


# model_name = 'gpt2'
# model = AutoModelForCausalLM.from_pretrained(model_name)
# ref_model = AutoModelForCausalLM.from_pretrained(model_name)
# value_model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

/Users/kakuayato/anaconda3/envs/trl_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(value_model)  # ここで None の場合、モデルが正しくロードされていない

AutoModelForCausalLMWithValueHead(
  (pretrained_model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2SdpaAttention(
            (c_attn): Conv1D(nf=2304, nx=768)
            (c_proj): Conv1D(nf=768, nx=768)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D(nf=3072, nx=768)
            (c_proj): Conv1D(nf=768, nx=3072)
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): 

## データセット作成

In [3]:
from datasets import Dataset

file_name = "test_train_data.txt" #改行区切りのテキストデータを用意
with open(file_name, "r", encoding="utf-8") as file:
    text_content = file.read()
text_data = text_content.split("\n")

#トークン化
tokenizer.pad_token = tokenizer.eos_token
tokenized_text = tokenizer(text_data, padding=True, truncation=True)
#学習のためDataset化
dataset = Dataset.from_dict(tokenized_text)

## LLM定義

In [4]:
import json

# 保存ファイル名
input_file = "x_y_data.json"

# JSONファイルを読み取る
with open(input_file, "r", encoding="utf-8") as file:
    data = json.load(file)

# データを変数に格納
x = data["x"]
y = data["y"]

print("データが読み取られました！")
print(f"x: {x[:3]}")  # xの最初の3要素を表示
print(f"y: {y[:3]}")  # yの最初の3要素を表示

データが読み取られました！
x: ['もちろんです！以下は、LLM（大規模言語モデル）を活用した新しいアプリのアイデアです。\n\n1. **パーソナライズド学習アプリ**:\n   学習者の理解度や興味に基づいて、カスタマイズされた学習プランや教材を提供するアプリ。ユーザーが質問をすると、LLMがそれに対する説明や関連資料を生成し、学習をサポートします。\n\n2. **ライティングアシスタント**:\n   ユーザーが書いた文章を分析し、文法やスタイルの改善点を提案するアプリ。さらに、特定のトピックについてのアイデアや構成を提供する機能も搭載。\n\n3. **メンタルヘルスサポートチャットボット**:\n   ユーザーが気軽に相談できるチャットボット。感情や悩みを共有すると、それに対する共感的な応答やリソースを提供し、必要に応じて専門家への相談を勧める機能を持つ。\n\n4. **旅行プランナー**:\n   ユーザーの好みや予算に基づいて、旅行の行き先やアクティビティを提案するアプリ。ユーザーが希望する条件を入力すると、LLMが最適なプランを生成します。\n\n5. **自動翻訳と文化ガイド**:\n   日常会話やビジネス文書をリアルタイムで翻訳するアプリ。翻訳だけでなく、文化的な背景や言語のニュアンスについても説明を加えることで、より深い理解を促します。\n\n6. **クリエイティブストーリー生成アプリ**:\n   ユーザーが入力したキーワードやテーマに基づいて、短編小説や詩を生成するアプリ。ユーザーが選んだスタイルやジャンルに応じて、異なるアプローチを提案します。\n\n7. **健康管理アシスタント**:\n   ユーザーの健康データ（食事、運動、睡眠など）を分析し、健康的な生活習慣を提案するアプリ。食事のレシピやエクササイズプランを生成し、モチベーションを高めるサポートをします。\n\n8. **パーソナルファイナンスアドバイザー**:\n   ユーザーの収入や支出に基づいて、予算管理や投資のアドバイスを提供するアプリ。LLMがユーザーの状況を理解し、具体的なアクションプランを提案します。\n\nこれらのアイデアは、LLMの能力を活かしてユーザーのニーズに応えるためのものです。どれか特に興味があるものがあれば、さらに詳しく考えてみることもできま

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

reward_base_model = make_pipeline(TfidfVectorizer(), MultinomialNB())
reward_base_model.fit(x, y)


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [6]:
# class RewardModel:
#     def __init__(self,reward_base_model,range_value=1):
#         self.reward_base_model = reward_base_model
#         self.range_value = range_value * 2 #報酬の値域を決める変数.-range_value<報酬の値<range_valueとなる。
        
#     def __call__(self,X):
#         output = self.reward_base_model.predict_proba(X)[:,1]
#         return (output-0.5) * self.range_value 

# reward_model = RewardModel(reward_base_model)


In [7]:
import torch.nn as nn
class RewardModel(nn.Module):
    def __init__(self, reward_base_model=None, range_value=1):
        super(RewardModel, self).__init__()
        self.reward_base_model = reward_base_model
        self.range_value = range_value * 2

    def forward(self, X):
        if self.reward_base_model is None:
            # 仮の報酬計算（デバッグ用）
            output = torch.rand(X.shape[0])  # ランダムな値を返す
        else:
            output = self.reward_base_model(X)
        return (output - 0.5) * self.range_value

reward_model = RewardModel(reward_base_model)

#### 報酬関数を使ってみる

## 学習

In [8]:
from transformers import GenerationConfig

# generation_configを手動で追加
model.generation_config = GenerationConfig()
ref_model.generation_config = GenerationConfig()

In [9]:
base_model = getattr(model, model.base_model_prefix)  # "bert"にアクセス
print(base_model)  # ベースのトランスフォーマーモデルが出力される

AttributeError: 'AutoModelForCausalLMWithValueHead' object has no attribute 'base_model_prefix'

In [11]:
from trl import PPOTrainer, PPOConfig
value_model.base_model_prefix = "transformer"
config = PPOConfig(batch_size=1, output_dir="./ppo_output") #今回はメモリサイズ短縮のためバッチサイズ1で行います。
ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    tokenizer=tokenizer,
    # その他の引数を必要に応じて追加
    ref_model=ref_model,
    reward_model=reward_model,
    train_dataset=dataset
)


/var/folders/s_/w4ncchnj77322v1x6dslwsq40000gn/T/ipykernel_86380/2842972833.py:4: FutureWarning: `config` is deprecated and removed starting from version 0.15.0 for `PPOTrainer.__init__`. Use `args` instead.
  ppo_trainer = PPOTrainer(
/Users/kakuayato/anaconda3/envs/trl_test/lib/python3.10/site-packages/transformers/utils/deprecation.py:165: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.15.0 for `PPOTrainer.__init__`. Use `processing_class` instead.
  return func(*args, **kwargs)


AttributeError: 'NoneType' object has no attribute 'base_model_prefix'

In [ ]:
#modelのテキスト生成時のオプション
generation_kwargs = {
    "top_k": 500,
    "top_p": 0.95,
    "min_length":50,
    "max_length": 100,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id
}

iteration = 0
for batch in dataset:
    query_tensors = [torch.tensor(batch['input_ids'])]

    #テキストの生成
    response = ppo_trainer.generate(query_tensors,**generation_kwargs)
    response_text = tokenizer.decode(response[0]) 
    
    #報酬の計算
    rewards = [torch.tensor(reward_model([response_text]))]
   
    #強化学習•重みの更新
    stats = ppo_trainer.step(query_tensors, response, rewards)
    iteration += 1